# # <div style="text-align:center"> Τεχνικές Εξόρυξης Δεδομένων: 2η Άσκηση  </div>

### <div style="text-align:center">  Παναγιώτοπουλος Γεώργιος **1115201700113** </div>

Αρχικά θα εισάγετε όλο το αρχείο σε ένα dataframe και θα μελετήσετε αν υπάρχουν τιμές Nan
στις στήλες ώστε να αφαιρέσετε αυτές τις γραμμες από το dataframe. Στην συνέχεια θα γράψετε
την κατάλληλες εντολές σε python για να απαντήσετε στα παρακάτω ζητούμενα. Τι περισσότερες
φορές οι απαντήσει θα δίνονται με ένα γράφημα και μπορείτε να χρησιμοποιήσετε όποια python
βιβλιοθήκη θέλετε για το σκοπό αυτό.

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('./books_1.Best_Books_Ever.csv', nrows=10000)
df.info(verbose=True)
print("Number of null values in 'title' column before cleaning:", df['series'].isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            10000 non-null  object 
 1   title             10000 non-null  object 
 2   series            5201 non-null   object 
 3   author            10000 non-null  object 
 4   rating            10000 non-null  float64
 5   description       9935 non-null   object 
 6   language          9700 non-null   object 
 7   isbn              10000 non-null  object 
 8   genres            10000 non-null  object 
 9   characters        10000 non-null  object 
 10  bookFormat        9919 non-null   object 
 11  edition           1357 non-null   object 
 12  pages             9875 non-null   object 
 13  publisher         9649 non-null   object 
 14  publishDate       9965 non-null   object 
 15  firstPublishDate  6528 non-null   object 
 16  awards            10000 non-null  object 

In [2]:
# Μπορούμε να βάλουμε τo median price σε όσα βιβλία δεν εχουν κάποια τίμη
df['price'] = df['price'].str.replace('.', '')
df['price'] = df['price'].astype(float)
price_median = df['price'].median()

# Επίσης μπορόυμε να υποθέσουμε ότι για τα βιβλία που δεν αναγράφεται edition,ισχύει το Standard edition
df['edition'].fillna('Standard Edition',inplace=True)

# Και ότι για όσα δεν αναγράφεται Series, ΄ότι είναι Standalone
df['series'].fillna('Standalone',inplace=True)

df_clean = df.dropna().reset_index(drop=True)
df_clean.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5414 entries, 0 to 5413
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            5414 non-null   object 
 1   title             5414 non-null   object 
 2   series            5414 non-null   object 
 3   author            5414 non-null   object 
 4   rating            5414 non-null   float64
 5   description       5414 non-null   object 
 6   language          5414 non-null   object 
 7   isbn              5414 non-null   object 
 8   genres            5414 non-null   object 
 9   characters        5414 non-null   object 
 10  bookFormat        5414 non-null   object 
 11  edition           5414 non-null   object 
 12  pages             5414 non-null   object 
 13  publisher         5414 non-null   object 
 14  publishDate       5414 non-null   object 
 15  firstPublishDate  5414 non-null   object 
 16  awards            5414 non-null   object 


C:\Users\north\AppData\Local\Temp\ipykernel_10096\3499395011.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] = df['price'].str.replace('.', '')


Έτσι φαίνεται να μπόρουμε να διατηρήσουμε ένα μεγάλο κομμάτι από το DataSet ```~70%```.

## 1) Προεπεξεργασία (10%) 
Παρατηρήστε την στήλη ratingsByStars, περιέχει 5 τιμές χωρισμένες με κόμματα , χωρίστε τις
τιμές αυτές και προσθέστε στο dataframe ξεχωριστά τα ratings, δηλαδή ratingStar5, ratingStar4,
ratingStar3 κτλ.
Επίσης η στήλη genres περιέχει για κάθε βιβλίο περισσότερα από ένα genre (είδος).
Δημιουργήστε μία νέα στήλη (ονομάστε την genreSingle) και βάλτε μόνο το πρώτο genre από
όλα τα genres που συναντάμε σε κάθε γραμμή. (πχ ['Fantasy', 'Young Adult', 'Fiction', 'Magic',
'Childrens', 'Adventure', 'Audiobook', 'Middle Grade', 'Classics', 'Science Fiction Fantasy'] -> η
νέα στήλη θα έχει το 'Fantasy' ) deleting books without any genre information.
Χρησιμοποιήστε την στήλη publishDate και δημιουργήστε μία νέα στήλη με το έτος έκδοσης κάθε
βιβλίου (μπορείτε να χρησιμοποιήστε την μέθοδο to_datetime() που παρέχει το pandas ή ότι
άλλο θέλετε) .

In [7]:
import numpy as np
import dateparser

#rows = len(df_clean.index)

# Useful function for getting rid of trash symbols
def remove_trash_symbols(string,symbols):
    for symbol in symbols:
            string = string.replace(symbol, "")
    return string

# Seperating rating entries
symbols_to_remove = " []''"
clean = []
reformatted = []
for i in range(len(df_clean.index)): # For the ratings of every book
    entry = df_clean['ratingsByStars'][i]
    formatted = list(entry.split(",")) # split the string in the comas
    for rating in formatted: # and for every element of the broken form ['1593642 left
        clean.append(int(remove_trash_symbols(rating,symbols_to_remove))) # Remove the extra symbols and turn the string into an int
reformatted = np.array(clean).reshape(len(df_clean.index),5)
# Adding the rows of the reformatted array into the dataframe
df_clean['ratingStar5'] = reformatted[:,0]
df_clean['ratingStar4'] = reformatted[:,1]
df_clean['ratingStar3'] = reformatted[:,2]
df_clean['ratingStar2'] = reformatted[:,3]
df_clean['ratingStar1'] = reformatted[:,4]

# Seperating genre
first_genre = []
for i in range(len(df_clean.index)): # For the ratings of every book
    entry = df_clean['genres'][i]
    first_genre.append(remove_trash_symbols(entry.split(',')[0],symbols_to_remove))
# Adding the new main genre column
df_clean['genreSingle'] = first_genre

years = []
for date in df_clean['publishDate']:
    parsed_date = dateparser.parse(date)
    if parsed_date is not None:
        years.append(parsed_date.year)
    else:
        years.append(None)  # or any other value to represent missing/invalid dates

df_clean['publishYear'] = years


df_clean.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5414 entries, 0 to 5413
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            5414 non-null   object 
 1   title             5414 non-null   object 
 2   series            5414 non-null   object 
 3   author            5414 non-null   object 
 4   rating            5414 non-null   float64
 5   description       5414 non-null   object 
 6   language          5414 non-null   object 
 7   isbn              5414 non-null   object 
 8   genres            5414 non-null   object 
 9   characters        5414 non-null   object 
 10  bookFormat        5414 non-null   object 
 11  edition           5414 non-null   object 
 12  pages             5414 non-null   object 
 13  publisher         5414 non-null   object 
 14  publishDate       5414 non-null   object 
 15  firstPublishDate  5414 non-null   object 
 16  awards            5414 non-null   object 


## 2) Ερωτήματα για μελέτη των δεδομένων - απαντήστε σε 5 από τα παρακάτω (20%)
1. Κατασκευάστε το ιστόγραμμα των ratings στο σύνολο δεδομένων (χρησιμοποιήστε την
στήλη rating)
2. Ποιά είναι τα 10 βιβλία με τις περισσότερες σελίδες.
3. Ποιά είναι τα 10 βιβλία με τα περισσότερα 5-αστέρια (χρησιμοποιήστε μόνο τα βιβλία που
έχουν λάβει πάνω από 10.000 5-star ratings από τη στήλη ratingStar5) .
4. Ποιές είναι οι πιο συχνές λέξεις στους τίτλους των βιβλίων (αφού αφαιρεθούν τα stop
words)
5. Ποιοι είναι οι 10 συγγραφεις με τα περισσότερα βιβλία
6. Ποιοι είναι οι 10 συγγραφείς με τις περισσότερες κριτικές (χρησιμοποιήστε την στήλη
numRatings).
7. Κατατάξτε του συγγραφείς με βάση τα βιβλία τους ανά έτος.
8. Ποιές είναι οι πιο συχνές γλώσσες που έχουν γραφτεί τα βιβλία στα δεδομένα σας
9. Ποιοί είναι οι 10 εκδότες που έχουν εκδώσει τα περισσότερα βιβλία.
10. Έχουν τα βιβλία με τις περισσότερες σελίδες (πχ περισσότερες από 1000 pages)
υψηλότερα ratings ?
11. Συγκεντρώστε σε ένα γράφημα η σε ένα πίνακα όλα τα μοναδικά είδη βιβλίων (genres).
Ποιά είναι τα πιο συχνά genres; Το ίδιο και για τα awards.
12. Φτιάξτε τα wordclouds για τη στήλη description. Σε αυτό το ερώτημα αφαιρέστε τα stop
words, πειραματιστείτε με τις παραμέτρους του wordcloud και εντοπίστε τις πιο
χαρακτηριστικές λέξεις που χρησιμοποιούνται στα βιβλία του συνόλου των δεδομένων
σας.
13. Πόσα βιβλία εκδίδονται ανά έτος ;

In [8]:
import matplotlib.pyplot as plt

# 2) Ποιά είναι τα 10 βιβλία με τις περισσότερες σελίδες.
sorted_by_pages = df_clean.sort_values('pages', key=lambda x: x)
print("Books with most pages: \n", sorted_by_pages['bookId'][:10])

# 3) Ποιά είναι τα 10 βιβλία με τις περισσότερες σελίδες.
sorted_by_rating = df_clean.sort_values('ratingStar5', key=lambda x: x)
print("Books with most 5 star ratings: \n", sorted_by_rating['bookId'][:10])

# 5) Ποιοι είναι οι 10 συγγραφεις με τα περισσότερα βιβλία
grouped_by_author = df_clean.groupby(['author']).count()
sorted_by_author_books = grouped_by_author.sort_values(['bookId'], key=lambda x: x,ascending=False)
print("Authors with most books: \n", sorted_by_author_books['bookId'][:10])

# 6) Ποιοι είναι οι 10 συγγραφείς με τις περισσότερες κριτικές
sorted_by_author_ratings = grouped_by_author.sort_values(['numRatings'], key=lambda x: x,ascending=False)
print("Authors with most ratings: \n", sorted_by_author_ratings['numRatings'][:10])

# 8) Ποιές είναι οι πιο συχνές γλώσσες που έχουν γραφτεί τα βιβλία
grouped_by_language = df_clean.groupby('language')['bookId'].count()
sorted_by_language = grouped_by_language.sort_values(ascending=False)
print("Most popular languages: \n", sorted_by_language[:10])


Books with most pages: 
 3142    492490.The_Song_of_the_Lioness_Quartet
2458                       30628.Space_Trilogy
1569              120461.Oscar_et_la_dame_rose
2995                       153492.The_39_Steps
746                     9539.The_Shadow_Rising
3865                    938544.Toll_the_Hounds
4490               92578.Wittgenstein_s_Nephew
3601                     821000.Now_We_Are_Six
969                        35231.Lord_of_Chaos
609                      153008.Kushiel_s_Dart
Name: bookId, dtype: object
Books with most 5 star ratings: 
 4541    8176759-a-terrace-on-the-tower-of-babel
3801                  35717446-one-day-you-will
3715           18281102-taming-the-blowing-wind
4538                    4934651-fe-fi-foe-comes
3141                    10757346-ain-t-no-grave
4598             17695243-call-of-the-lost-ages
4609          15733160-meditation-on-space-time
2729                           13638968-undying
2730           14655460-as-the-matzo-ball-turns
4549       

## 3) Υλοποίηση Recommendation system (35%)
Ο στόχος ενός τέτοιου συστήματος είναι (1) να προβλέψει τις αξιολογήσεις ενός χρήστη για τα
βιβλία που δεν έχει διαβάσει ακόμα, και (2) να εμφανίσει ένα ταξινομημένη λίστα με τα κορυφαία
Ν βιβλία για τα οποία πιστεύουμε ότι θα ήθελαν να μάθουν περισσότερα. ‘Ενας άλλος στόχος
ενός Recommender είναι (3) να βοηθήσει τους χρήστες να ανακαλύψουν σχετικά βιβλία που δεν
θα είχαν βρει διαφορετικά.
Σε αυτό το ερώτημα θα χρειαστείτε τις στήλες
* BookId
* Description
* Και μόνο όσες γραμμές έχουν γλώσσα “English”. 

Δημιουργήστε τον TF-IDF (Term Frequency - Inverse Document Frequency) πίνακα των
unigrams και των bigrams για τη στήλη description (χρησιμοποιήστε την παράμετρο stop_word
του TfidfVectorizer).

**Cosine Similarity**: Η μετρική αυτή υπολογίζει την ομοιότητα μεταξύ δύο διανυσμάτων x,y,
χρησιμοποιώντας τη γωνία μεταξύ τους (όταν η γωνία είναι 0 σημαίνει ότι τα x και y είναι ίσα , αν
εξαιρέσουμε το μήκος τους). Διατρέξτε τον TF-IDF πίνακα και υπολογίστε το similarity καθενός
βιβλίου με τα υπόλοιπα. Αποθηκεύστε σε ένα python dictionary τα 100 πιο όμοια βιβλία.

Πρόβλεψη: Φτιάξτε μία συνάρτηση η οποία παίρνει σαν είσοδο ένα id και ένα ακέραιο αριθμό N,
και επιστρέφει τα Ν πιο όμοια βιβλία.
recommend(item_id = 4085439, num = 5)

Η έξοδος της συνάρτησης να είναι της παρακάτω μορφής μορφής:
```
Recommending 5 books similar to: The Hunger Games
-------------------------------------------------
Recommended: NAME
Description: DESCRIPTION
(score:0.12235188993161432)
Recommended: NAME
Description: DESCRIPTION
(score:0.12235188993161432 
```


In [9]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Keep only the book Ids and their respective descriptions 
book_descriptions = df_clean.loc[
    (df_clean['language'] == 'English'),['bookId','description']].reset_index(drop=True)

vectorizer = TfidfVectorizer(
    token_pattern = r'\b(?![A-Za-z]\b)[A-Za-z]+\b', #ignore numbers and such
    #min_df = 0.0014, # I exclude rare words, as they have little contribution to overarching theme of a book,might change it later
    ngram_range=(1, 2), # unigrams and bigrams
    strip_accents='unicode', # probably won't need that since we are only in English but you never know
    stop_words='english', # ignore stop words 
)

#rows = len(book_descriptions.index)

matrix = vectorizer.fit_transform(book_descriptions['description'])

sims = cosine_similarity(matrix) # calculate the similarity matrix (735,735)
np.fill_diagonal(sims,-1) # remove the self similarities
flat = sims.flatten()
# creating pairs with the 100 most similar books
pairs = []
for i in range(100): # for 100 times
    index = np.argmax(flat) # get the largest remaining similarity in the flat matrix
    flat[index] = -1 # get remove it from the matrix so as to not find it again
    row, column = divmod(index,len(book_descriptions.index)) # calculate the row and column
    symmetrical_index = column*len(book_descriptions.index) + row # calculate the symmetrical case index so we can remove it as well
    flat[symmetrical_index] = -1 # remove the symmetrical case also
    pairs.append((row,column)) # save the similarity pair
    
# creating a dictionary with the 100 most similar books
dictionary = {}
i = 0
for (index1,index2) in pairs:
    dictionary[i] = (book_descriptions['bookId'][index1],book_descriptions['bookId'][index2])
    i = i+1

def recommend(bookId,findNsimilar): # needs the book_descriptions to have been calculated 
    print("Recommending 5 books similar to: ", bookId,"\n-------------------------------------------------")
    similars = [] # the list of the indices of the similar books
    row = (book_descriptions[
        book_descriptions['bookId'] == bookId].index.tolist())[0] # get the row of the book 
    array = sims[row] # get the array of the similarities
    for i in range(findNsimilar):
        index = np.argmax(array) # get the index of the similar books
        similars.append((index,array[index],book_descriptions['bookId'][index])) # append entry 
        print("Recommended: ",book_descriptions['bookId'][index])
        print("Description: ",book_descriptions['description'][index])
        print("(score: ",array[index],")\n")
        array[index] = -1 # don't take it into account again
    return similars

recommend('4948.The_Very_Hungry_Caterpillar',5)
    

Recommending 5 books similar to:  4948.The_Very_Hungry_Caterpillar 
-------------------------------------------------
Recommended:  16074809-letting-ana-go
Description:  In the tradition of Go Ask Alice and Lucy in the Sky, a harrowing account of anorexia and addiction.She was a good girl from a good family, with everything she could want or need. But below the surface, she felt like she could never be good enough. Like she could never live up to the expectations that surrounded her. Like she couldn’t do anything to make a change.But there was one thing she could control completely: how much she ate. The less she ate, the better—stronger—she felt.But it’s a dangerous game, and there is such a thing as going too far…Her innermost thoughts and feelings are chronicled in the diary she left behind.
(score:  0.04239881532429841 )

Recommended:  759611.Brown_Bear_Brown_Bear_What_Do_You_See_
Description:  A big happy frog, a plump purple cat, a handsome blue horse, and a soft yellow duck--all

[(3221, 0.04239881532429841, '16074809-letting-ana-go'),
 (4069, 0.036528702246348535, '759611.Brown_Bear_Brown_Bear_What_Do_You_See_'),
 (4750, 0.034510136485829346, '10393675-until-tuesday'),
 (1075, 0.03380925147361972, '92160.London'),
 (676, 0.033309905306431624, '34053.Just_So_Stories')]

## 4) Υλοποίηση Κατηγοριοποίησης (Classification) (35%)
Χρησιμοποιήστε την στήλη genreSingle, βρείτε τα 10 πιο συχνά genres και κρατήστε σε ένα νέο
dataframe τα βιβλία εκείνα που ανήκουν σε αυτες τις 10 πιο συχνές κατηγορίες. Θα χρειαστείτε
το bookId το description και το genreSingle. Στην συνέχεια καθαρίστε την στήλη description
χρησιμοποιώντας τις μεθόδους που είδαμε στα φροντιστήρια (πχ αφαίρεση σημείων στίξης,
μετατροπή όλων των χαρακτήρων σε πεζά, κ.α.). Εφαρμόστε την μέθοδο word2vec για τα
descriptions και στην συνέχεια με την χρήση των embeddings να υπολογίσετε για κάθε
description ένα διάνυσμα με 200-300 τιμές (features) - αυτό θα είναι ο μέσος όρος των
embeddings των λέξεων από τις οποίες αποτελείται το description.

Χρησιμοποιήστε τη βιβλιοθήκη pickle της Python για να αποθηκεύσετε τα χαρακτηριστικά σε
αρχεία *.pkl . Με αυτό τον τρόπο δεν χρειάζεται να υπολογίζονται από την αρχή τα
χαρακτηριστικά κάθε φορά που τρέχετε το πρόγραμμά σας, αλλά μπορείτε μόνο να τα φορτωνεται
στην μνήμη χρησιμοποιώντας την αντίστοιχη μέθοδο load.
Χωρίστε το σύνολο των δεδομένων σε train (80%) και test (20%) χρησιμοποιώντας την μέθοδο
train_test_split() της βιβλιοθήκης sklearn.

Σε αυτό το ερώτημα θα πρέπει το πρόγραμμα σας να μπορεί να βρει τις κατηγορίες (genre) του
συνόλου δοκιμής (test) χρησιμοποιώντας τις παρακάτω μεθόδους Classification:
* Naive Bayes
* Support Vector Machines (SVM, να πειραματιστείτε με τις παραμέτρους kernel (rbf, linear), c
και gamma. H επιλογή των παραμέτρων μπορεί να γίνει και με GridSearchCV)
* Random Forests

Ολα τα παραπάνω μοντέλα θα εκπαιδευτούν MONO στο σύνολο train και θα αξιολογηθούν στο
σύνολο test. Επίσης θα πρέπει να αξιολογήσετε και να καταγράψετε την απόδοση κάθε μεθόδου
χρησιμοποιώντας 10-fold Cross Validation χρησιμοποιώντας τις παρακάτω μετρικές:
* Precision / Recall / F-Measure
* Accuracy

Στο τέλος να ετοιμάσετε ένα πίνακα με τα αποτελέσματα των πειραμάτων σας για κάθε
μετρική/παράμετρο που χρησιμοποιήσατε.